# Named Entity Recognition (NER) using BERT



# Install Transformers and Datasets from Hugging Face

In [1]:
# Transformers installation
! pip install -q transformers datasets evaluate seqeval accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


## Load the WNUT 2017 dataset


In [2]:
from datasets import load_dataset

wnut = load_dataset('wnut_17', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

# List the Tag Names in the WNUT 2017 Dataset

In [3]:
tag_names = wnut["test"].features["ner_tags"].feature.names
tag_names

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

# Load the Toknenizer of the bert-base-NER Model

In [4]:
from transformers import AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Tokenize the WNUT 17 Dataset

In [5]:
def tokenize_and_align_tags(records):
    # Tokenize the input words. This will break words into subtokens if necessary.
    # For instance, "ChatGPT" might become ["Chat", "##G", "##PT"].
    tokenized_results = tokenizer(records["tokens"], truncation=True, is_split_into_words=True)

    input_tags_list = []

    # Iterate through each set of tags in the records.
    for i, given_tags in enumerate(records["ner_tags"]):
        # Get the word IDs corresponding to each token. This tells us to which original word each token corresponds.
        word_ids = tokenized_results.word_ids(batch_index=i)

        previous_word_id = None
        input_tags = []

        # For each token, determine which tag it should get.
        for wid in word_ids:
            # If the token does not correspond to any word (e.g., it's a special token), set its tag to -100.
            if wid is None:
                input_tags.append(-100)
            # If the token corresponds to a new word, use the tag for that word.
            elif wid != previous_word_id:
                input_tags.append(given_tags[wid])
            # If the token is a subtoken (i.e., part of a word we've already tagged), set its tag to -100.
            else:
                input_tags.append(-100)
            previous_word_id = wid

        input_tags_list.append(input_tags)

    # Add the assigned tags to the tokenized results.
    # Hagging Face trasformers use 'labels' parameter in a dataset to compute losses.
    tokenized_results["labels"] = input_tags_list

    return tokenized_results


In [6]:
tokenized_wnut = wnut.map(tokenize_and_align_tags, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [11]:
for key in wnut['train'][1]:
    print(key, ":", wnut['train'][2][key])

id : 2
tokens : ['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']
ner_tags : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
for key in tokenized_wnut['train'][1]:
    print(key, ":", tokenized_wnut['train'][2][key])

id : 2
tokens : ['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']
ner_tags : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_ids : [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels : [-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 0, -100, -100]


# Create Data Collator

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Fine Tune the bert-base-NER Model
The above steps prepared the WNUT datasets. Now, let us fine tine the bert-base-NER model.

## Create Maps from Tag Ids to Tag Names

Before we start fine-tuning the model, we create a map of the expected tag ids to their tag names with `id2label` and `label2id`:

In [14]:
id2label = dict(enumerate(tag_names))
id2label

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

In [15]:
label2id = dict(zip(id2label.values(), id2label.keys()))
label2id

{'O': 0,
 'B-corporation': 1,
 'I-corporation': 2,
 'B-creative-work': 3,
 'I-creative-work': 4,
 'B-group': 5,
 'I-group': 6,
 'B-location': 7,
 'I-location': 8,
 'B-person': 9,
 'I-person': 10,
 'B-product': 11,
 'I-product': 12}

## Load the Pre-Trained Model by Passing the Maps between Ids and Names
Load the bert-base-NER model with AutoModelForTokenClassification along with the number of expected tags, and the tag mappings:

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(id2label), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Customized Evaluation Metrics

To better evaluate the model's performance during training, we should pass our own evaluation metrics.

You will leverage the Huggingface [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, we will load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the Huggingface Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric).

Seqeval produces several scores: precision, recall, F1, and accuracy.

Install evaluate and seqeval.

Import evaluate and create a seqeval instance.

In [17]:
import evaluate

seqeval = evaluate.load("seqeval")

Define a function `compute_metrics` that can be used in Trainer. The function will receive the results generated by the model in a batch mode. We will unpack the results and compute the metrics.

In [18]:
import numpy as np

def compute_metrics(p):
    # p is the results containing a list of predictions and a list of labels
    # Unpack the predictions and true labels from the input tuple 'p'.
    predictions_list, labels_list = p

    # Convert the raw prediction scores into tag indices by selecting the tag with the highest score for each token.
    predictions_list = np.argmax(predictions_list, axis=2)

    # Filter out the '-100' labels that were used to ignore certain tokens (like sub-tokens or special tokens).
    # Convert the numeric tags in 'predictions' and 'labels' back to their string representation using 'tag_names'.
    # Only consider tokens that have tags different from '-100'.
    true_predictions = [
        [tag_names[p] for (p, l) in zip(predictions, labels) if l != -100]
        for predictions, labels in zip(predictions_list, labels_list)
    ]
    true_tags = [
        [tag_names[l] for (p, l) in zip(predictions, labels) if l != -100]
        for predictions, labels in zip(predictions_list, labels_list)
    ]

    # Evaluate the predictions using the 'seqeval' library, which is commonly used for sequence labeling tasks like NER.
    # This provides metrics like precision, recall, and F1 score for sequence labeling tasks.
    results = seqeval.compute(predictions=true_predictions, references=true_tags)

    # Return the evaluated metrics as a dictionary.
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


## Define TrainingArguments

Define a TrainingArguments instance with different parameters.

In [26]:
training_args = TrainingArguments(
    output_dir="my_finetuned_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Let us fine tune it with the metrics and training parameters.

In [27]:
import numpy as np

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.248590,0.575403,0.364226,0.446084,0.943654
2,No log,0.271401,0.562932,0.377201,0.451720,0.945748


TrainOutput(global_step=426, training_loss=0.10058069900727608, metrics={'train_runtime': 114.7035, 'train_samples_per_second': 59.179, 'train_steps_per_second': 3.714, 'total_flos': 183537374335236.0, 'train_loss': 0.10058069900727608, 'epoch': 2.0})

## Inference
We can do an inference as before by loading the fine-tuned model from the hub.

In [30]:
text = wnut['test'][0]['tokens']
text

['&',
 'gt',
 ';',
 '*',
 'The',
 'soldier',
 'was',
 'killed',
 'when',
 'another',
 'avalanche',
 'hit',
 'an',
 'army',
 'barracks',
 'in',
 'the',
 'northern',
 'area',
 'of',
 'Sonmarg',
 ',',
 'said',
 'a',
 'military',
 'spokesman',
 '.']

Load the fine tuned model and tokenizer from the hub.

In [28]:
model = AutoModelForTokenClassification.from_pretrained("my_finetuned_wnut_model/checkpoint-426")
tokenizer = AutoTokenizer.from_pretrained("my_finetuned_wnut_model/checkpoint-426")

classifier = pipeline("ner", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Classify the text.

In [31]:
_sentence = " ".join(text)
print(_sentence)
classifier(_sentence)

& gt ; * The soldier was killed when another avalanche hit an army barracks in the northern area of Sonmarg , said a military spokesman .


[{'entity': 'B-location',
  'score': 0.6833235,
  'index': 21,
  'word': 'son',
  'start': 100,
  'end': 103},
 {'entity': 'I-location',
  'score': 0.4762397,
  'index': 22,
  'word': '##mar',
  'start': 103,
  'end': 106},
 {'entity': 'I-location',
  'score': 0.4706667,
  'index': 23,
  'word': '##g',
  'start': 106,
  'end': 107}]

In [34]:
gt = wnut['test'][0]['ner_tags']
for i, t in enumerate(text):
    print(t, tag_names[gt[i]])

& O
gt O
; O
* O
The O
soldier O
was O
killed O
when O
another O
avalanche O
hit O
an O
army O
barracks O
in O
the O
northern O
area O
of O
Sonmarg B-location
, O
said O
a O
military O
spokesman O
. O
